In [584]:
%run -n main.py
lines = read_lines(DOTENV_PATH)
pairs = parse_dotenv(lines)
os.environ.update(pairs)
%run -n main.py

# tasks

## terra

In [ ]:
path = 'data/rsg/TERRa/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] == 'entailment'], 50)
    + random.sample([_ for _ in items if _['label'] == 'not_entailment'], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/terra.jsonl', lines)

## danetqa

In [ ]:
path = 'data/rsg/DaNetQA/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] is True], 50)
    + random.sample([_ for _ in items if _['label'] is False], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/danetqa.jsonl', lines)

## parus

In [ ]:
path = 'data/rsg/PARus/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['question'] == 'effect'], 48)
    + random.sample([_ for _ in items if _['question'] == 'cause'], 52)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/parus.jsonl', lines)

## rwsd

In [ ]:
path = 'data/rsg/RWSD/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] == True], 50)
    + random.sample([_ for _ in items if _['label'] == False], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/rwsd.jsonl', lines)

## russe

In [ ]:
path = 'data/rsg/RUSSE/val.jsonl'
lines = read_lines(path)
items = list(parse_jsonl(lines))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['label'] == True], 50)
    + random.sample([_ for _ in items if _['label'] == False], 50)
)
random.shuffle(items)
for item in items:
    item['id'] = item.pop('idx')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/russe.jsonl', lines)

## rucola

In [ ]:
path = 'data/rucola/out_of_domain_dev.csv'
items = list(read_csv(path))

In [ ]:
random.seed(0)
items = (
    random.sample([_ for _ in items if _['acceptable'] == '0' and _['error_type'] == 'Hallucination'], 50)
    + random.sample([_ for _ in items if _['acceptable'] == '1'], 50)
)
random.shuffle(items)
for item in items:
    item['label'] = item.pop('acceptable')

In [ ]:
lines = format_jsonl(items)
write_lines('tasks/rucola.jsonl', lines)

# eval apis

In [581]:
%run -n main.py
task = RUCOLA
eval = f'17_curie_{task}'

In [582]:
%run -n main.py
lines = read_lines(f'tasks/{task}.jsonl')
task_items = list(parse_jsonl(lines))

In [583]:
%run -n main.py
task_item = random.choice(task_items)
prompt = TASK_PROMPTS[task](task_item)
print(prompt)

Предложение корректное или нет? Проверь синтаксис, семантику и морфологию.
Ответь коротко: да или нет.
---
Предложение: Ты сидела слишком близко от него.
Корректное: Да
---
Предложение: Я слышал вой и лай собак и радовался, воображая, что ехать неподалеку.
Корректное: Нет
---
Предложение: Он мне сказал, что приходи.
Корректное: Нет
---
Предложение: А ты ехай прямо к директору театров, князю Гагарину.
Корректное: Нет
---
Предложение: Те, кто внимательно прочитает заглавие статьи, может определить и тему, и основную мысль текста.
Корректное: 


In [ ]:
%run -n main.py
response = openai_completions(prompt, model=TEXT_CURIE_001, stop='---')
print(response)

In [ ]:
path = Path(f'evals/{eval}.jsonl')
eval_items = []
if path.exists():
    lines = read_lines(path)
    eval_items.extend(parse_jsonl(lines))
len(eval_items)

In [ ]:
%run -n main.py
cache_ids = {_['id'] for _ in eval_items}
nocache_task_items = [_ for _ in task_items if _['id'] not in cache_ids]

for task_item in log_progress(nocache_task_items):
    prompt = TASK_PROMPTS[task](task_item)

    response = openai_completions(prompt, model=TEXT_CURIE_001, stop='---')
    sleep(2)

    # response = openai_chat_completions(prompt, model=GPT_35_TURBO_0301, stop='---')
    # sleep(2)

    # response = cohere_generate(prompt, end_sequences=['---'])
    # sleep(12)

    eval_items.append({
        'id': task_item['id'],
        'response': response
    })


In [ ]:
eval_items

In [ ]:
[_['label'] for _ in task_items[:10]]

In [ ]:
lines = format_jsonl(eval_items)
write_lines(f'evals/{eval}.jsonl', lines)

# score

In [599]:
%run -n main.py
model_task_scores = []
for (model, task), eval in MODEL_TASK_EVALS.items():
    lines = read_lines(f'tasks/{task}.jsonl')
    id_targets = {
        _['id']: _['label']
        for _ in parse_jsonl(lines)
    }
    
    lines = read_lines(f'evals/{eval}.jsonl')
    norm_response = NORM_RESPONSES[task]
    id_preds = {
        _['id']: norm_response(_['response'])
        for _ in parse_jsonl(lines)
    }
    score = acc_score(id_targets, id_preds)
    model_task_scores.append((model, task, score))

table = scores_table(model_task_scores)
table

task,terra,danetqa,parus,rwsd,russe,rucola
model,,,,,,
human,0.92,0.915,0.982,0.84,0.805,0.84
sota,0.877,0.917,0.908,0.675,0.823,0.82
openai/turbo,0.86,"0.82, 3!","0.91, 8!",0.79,0.68,0.53
openai/davinci,0.91,0.79,0.93,-,-,-
openai/curie,0.57,"0.71, 1!",0.60,-,-,-
cohere/xlarge,0.72,0.50,0.54,-,-,-
sberbank-ai/rugpt3small_based_on_gpt2,"0.66, 71!","0.61, 36!","0.51, 51!","0.32, 59!","0.57, 54!","0.48, 46!"
sberbank-ai/rugpt3medium_based_on_gpt2,"0.48, 25!","0.54, 28!","0.70, 67!","0.55, 16!","0.49, 17!","0.51, 14!"
sberbank-ai/rugpt3large_based_on_gpt2,"0.59, 56!","0.51, 27!","0.51, 63!","0.36, 34!","0.52, 42!","0.51, 55!"
